In [1]:
import xlrd
import pandas as pd
import csv
import sqlite3
import matplotlib
matplotlib.use("svg")
import matplotlib.pyplot as plt
import os
if not os.environ.get("DISABLE_TESTING", False):
    %matplotlib inline
plt.style.use('ggplot')
import numpy as np

In [2]:
##Read table using pandas 
df1 = pd.read_csv("grad_rates_pupil_teacher_ratio.csv",skiprows=6)

In [3]:
df1

,County Name,County Name [Public School] 2017-18,County Name [Public School] 2016-17,Total Number of Public Schools [Public School] 2017-18,Total Number of Public Schools [Public School] 2016-17,American Indian/Alaska Native - male [Public School] 2017-18,American Indian/Alaska Native - male [Public School] 2016-17,American Indian/Alaska Native - female [Public School] 2017-18,American Indian/Alaska Native - female [Public School] 2016-17,Asian or Asian/Pacific Islander - male [Public School] 2017-18,...,Hawaiian Nat./Pacific Isl. - male [Public School] 2017-18,Hawaiian Nat./Pacific Isl. - male [Public School] 2016-17,Hawaiian Nat./Pacific Isl. - female [Public School] 2017-18,Hawaiian Nat./Pacific Isl. - female [Public School] 2016-17,Two or More Races - male [Public School] 2017-18,Two or More Races - male [Public School] 2016-17,Two or More Races - female [Public School] 2017-18,Two or More Races - female [Public School] 2016-17,Pupil/Teacher Ratio [Public School] 2017-18,Pupil/Teacher Ratio [Public School] 2016-17
0,Abbeville County SC,Abbeville County,Abbeville County,10.0,10.0,3,3,3,1,4,...,3,2,1,"=""0""",34,38,26,28,13.17,13.49
1,Acadia Parish LA,Acadia Parish,Acadia Parish,27.0,27.0,†,†,†,†,†,...,†,†,†,†,207,198,237,214,20.30,17.92
2,Accomack County VA,Accomack County,Accomack County,13.0,13.0,4,4,4,4,22,...,"=""0""",1,5,4,15,16,18,16,16.55,16.26
3,Ada County ID,Ada County,Ada County,136.0,134.0,210,224,207,221,991,...,†,163,†,166,1446,1313,1358,1240,18.99,19.25
4,Adair County IA,Adair County,Adair County,5.0,5.0,–,–,†,†,†,...,–,–,–,–,12,13,14,11,11.41,12.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,Totals:,NaN,NaN,99337.0,99138.0,†,†,†,†,†,...,†,†,†,†,952629,890554,924383,860392,NaN,NaN
3139,Data Source: U.S. Department of Education Nati...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3140,† indicates that the data are not applicable.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3141,– indicates that the data are missing.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#create and read table using sqlite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE GRAD_RATIO(
    county_name TEXT,
    county_name_17 TEXT,
    county_name_16 TEXT,
    total_num_schools_17 INTEGER,
    total_num_schools_16 INTEGER,
    american_indian_male_17 INTEGER,
    american_indian_male_16 INTEGER,
    american_indian_female_17 INTEGER,
    american_indian_female_16 INTEGER,
    asian_male_17 INTEGER,
    asian_male_16 INTEGER,
    asian_female_17 INTEGER,
    asian_female_16 INTEGER,
    hispanic_male_17 INTEGER,
    hispanic_male_16 INTEGER,
    hispanic_female_17 INTEGER,
    hispanic_female_16 INTEGER,
    black_male_17 INTEGER,
    black_male_16 INTEGER,
    black_female_17 INTEGER,
    black_female_16 INTEGER,
    white_male_17 INTEGER,
    white_male_16 INTEGER,
    white_female_17 INTEGER,
    white_female_16 INTEGER,
    hawaiian_male_17 INTEGER,
    hawaiian_male_16 INTEGER,
    hawaiian_female_17 INTEGER,
    hawaiian_female_16 INTEGER,
    mixed_male_17 INTEGER,
    mixed_male_16 INTEGER,
    mixed_female_17 INTEGER,
    mixed_female_16 INTEGER,
    student_teacher_ratio_17 INTEGER,
    student_teacher_ratio_16 INTEGER
);""")

conn.commit()

with open ("grad_rates_pupil_teacher_ratio.csv") as grad_ratio:
    data = csv.reader(grad_ratio)
    counter = 0
    for row in data:
        if counter >= 7 and len(row)>=35:
            #print(counter)
            cursor.execute("""INSERT INTO GRAD_RATIO VALUES (?,?,?,?,?,?,?,?,?,?,
                                                             ?,?,?,?,?,?,?,?,?,?,
                                                             ?,?,?,?,?,?,?,?,?,?,
                                                             ?,?,?,?,?)""", tuple(row))
        counter += 1

        conn.commit()
            

pd.read_sql_query("SELECT * from GRAD_RATIO;", conn)

,county_name,county_name_17,county_name_16,total_num_schools_17,total_num_schools_16,american_indian_male_17,american_indian_male_16,american_indian_female_17,american_indian_female_16,asian_male_17,...,hawaiian_male_17,hawaiian_male_16,hawaiian_female_17,hawaiian_female_16,mixed_male_17,mixed_male_16,mixed_female_17,mixed_female_16,student_teacher_ratio_17,student_teacher_ratio_16
0,Abbeville County SC,Abbeville County,Abbeville County,10,10,3,3,3,1,4,...,3,2,1,"=""0""",34,38,26,28,13.17,13.49
1,Acadia Parish LA,Acadia Parish,Acadia Parish,27,27,†,†,†,†,†,...,†,†,†,†,207,198,237,214,20.3,17.92
2,Accomack County VA,Accomack County,Accomack County,13,13,4,4,4,4,22,...,"=""0""",1,5,4,15,16,18,16,16.55,16.26
3,Ada County ID,Ada County,Ada County,136,134,210,224,207,221,991,...,†,163,†,166,1446,1313,1358,1240,18.99,19.25
4,Adair County IA,Adair County,Adair County,5,5,–,–,†,†,†,...,–,–,–,–,12,13,14,11,11.41,12.28
5,Adair County KY,Adair County,Adair County,5,5,†,†,†,†,5,...,†,†,†,†,59,48,45,41,15.53,14.86
6,Adair County MO,Adair County,Adair County,9,9,†,†,†,†,†,...,†,†,†,†,50,†,55,†,12.28,12.6
7,Adair County OK,Adair County,Adair County,18,18,1476,1520,1466,1479,†,...,†,1,–,"=""0""",112,90,122,96,13.28,13.41
8,Adams County CO,Adams County,Adams County,157,158,†,†,†,†,1710,...,†,†,†,†,1161,1097,1147,1114,20.17,19.83
9,Adams County IA,Adams County,Adams County,2,2,†,–,†,†,†,...,–,–,–,–,5,5,†,†,10.78,11.42


In [30]:
fin_df = pd.read_excel("financial_data.xls", sheet_name=["14","15","16","17","18"], skiprow=5)

In [82]:
fin_df["14"]

,Table 14.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Per Pupil Amounts of Public Elementary-Secondary School System Finances by,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,Enrollment-Size Groups: Fiscal Year 2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(Dollars. Detail may not add to total because ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Item,NaN,NaN,NaN,NaN,NaN,School systems with enrollment of:,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,All school,"50,000 or","25,000 to","15,000 to","7,500 to","5,000 to","3,000 to",under
6,NaN,NaN,NaN,NaN,NaN,systems,more,49999,24999,14999,7499,4999,3000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Fall enrollment, 2016............................",NaN,NaN,NaN,NaN,48631444,10735685,6648975,5146011,7325603,4156922,5273523,9344725
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# print(fin_df["14"].columns)
# print(fin_df["14"].columns[0])
# print(fin_df["14"][fin_df["14"].columns[0]])



for i in range(4):
    print(fin_df["14"].columns[i])
    col_data = fin_df["14"][fin_df["14"].columns[i]]

Table 14. 
Unnamed: 1
Unnamed: 2
Unnamed: 3


In [161]:
fin_df_pps = fin_df["18"].fillna(-1)
fin_df_pps

,Table 18. Per Pupil Amounts for Current Spending of the 100 Largest Public Elementary-Secondary School Systems in the United,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,States by Enrollment: Fiscal Year 2017,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,(Dollars. Detail may not add to total because ...,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,Rank,-1.0,School System,State,-1,Current spending,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1.0,-1,-1,-1,-1,For selected objects,-1,For selected functions,-1,-1,-1,-1,-1,-1,-1
5,-1,-1.0,-1,-1,-1,-1,-1,-1,Instruction,-1,-1,Support services,-1,-1,-1,-1
6,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,Instructional,General,School
7,-1,-1.0,-1,-1,Enroll-,-1,Salaries and,Employee,-1,Salaries and,Employee,-1,Pupil,staff,adminis-,adminis-
8,-1,-1.0,-1,-1,ment1,Total2,wages,benefits,Total2,wages,benefits,Total2,support,support,tration,tration
9,-1,-1.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [165]:
# print(fin_df["18"].columns)
per_pupil_spending = pd.DataFrame([], columns=["Rank", "School System", "State", "Enrollment", "Instruction_Total", 
                              "Instruction_Salaries", "Instruction_EmployeeBenefits", "Support_Total", 
                              "Support_Pupil", "Support_Staff","Support_GeneralAdmin", "Support_SchoolAdmin"])
per_pupil_spending["Rank"] = [i for i in range(100)]
per_pupil_spending["School System"] = [i for i in fin_df_pps["Unnamed: 2"][4:] if i !="School System" and i != -1]
per_pupil_spending["State"] = [i for i in fin_df_pps["Unnamed: 3"][4:] if i !="State" and i != -1]
per_pupil_spending["Enrollment"] = [int(i) for i in fin_df_pps["Unnamed: 4"] if i != -1 and type(i) != str]
per_pupil_spending["Instruction_Total"] = [int(i) for i in fin_df_pps["Unnamed: 8"] if i != -1 and type(i) != str]
per_pupil_spending["Instruction_Salaries"] = [int(i) for i in fin_df_pps["Unnamed: 9"] if i != -1 and type(i) != str]
per_pupil_spending["Instruction_EmployeeBenefits"] = [int(i) for i in fin_df_pps["Unnamed: 10"] if i != -1 and type(i) != str]
per_pupil_spending["Support_Total"] = [int(i) for i in fin_df_pps["Unnamed: 11"] if i != -1 and type(i) != str]
per_pupil_spending["Support_Pupil"] = [int(i) for i in fin_df_pps["Unnamed: 12"] if i != -1 and type(i) != str]
per_pupil_spending["Support_Staff"] = [int(i) for i in fin_df_pps["Unnamed: 13"] if i != -1 and type(i) != str]
per_pupil_spending["Support_GeneralAdmin"] = [int(i) for i in fin_df_pps["Unnamed: 14"] if i != -1 and type(i) != str]
per_pupil_spending["Support_SchoolAdmin"] = [int(i) for i in fin_df_pps["Unnamed: 15"] if i != -1 and type(i) != str]

per_pupil_spending.to_csv(index=False)

    Rank           School System           State  Enrollment  \
0      0    New York City County        New York      984462   
1      1     Los Angeles Unified      California      633621   
2      2             Cook County        Illinois      378199   
3      3       Miami-Dade County         Florida      357249   
4      4            Clark County          Nevada      326953   
5      5          Broward County         Florida      271852   
6      6          Houston County           Texas      216106   
7      7     Hillsborough County         Florida      214386   
8      8           Orange County         Florida      200674   
9      9       Palm Beach County         Florida      192721   
10    10          Fairfax County        Virginia      187467   
11    11   Hawaii Public Schools          Hawaii      181550   
12    12         Gwinnett County         Georgia      178214   
13    13             Wake County  North Carolina      160467   
14    14       Montgomery County        